In [30]:
from dataclasses     import dataclass
from typing          import Dict, List, Tuple
import numpy         as np
from qiskit.quantum_info import Pauli, Statevector
from qiskit          import QuantumCircuit
from collections     import defaultdict
import random
from qiskit.circuit.library import UnitaryGate  

from pauli_propagation.utils import random_su4

In [ ]:
# Import Qiskit Library
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, Statevector
from qiskit.circuit.library import UnitaryGate  

from pauli_propagation import PauliTerm, PauliPropagator
from pauli_propagation.utils import random_su4

import matplotlib.pyplot as plt
import logging
import numpy as np
import random

logging.getLogger("matplotlib.font_manager").setLevel(logging.ERROR)

In [9]:

def _staircase_edges(nx, ny):
    """Return the ordered list of 1-based index pairs of the staircase walk."""
    next_inds, temp_inds, edges = [1], [], []
    while next_inds:
        for ind in next_inds:
            if ind % nx != 0:                      # step right
                nxt = ind + 1
                edges.append((ind, nxt)); temp_inds.append(nxt)
            if ((ind-1)//nx + 1) < ny:             # step down
                nxt = ind + nx
                edges.append((ind, nxt)); temp_inds.append(nxt)
        next_inds, temp_inds = temp_inds, []
    seen, uniq = set(), []
    for e in edges:                                # preserve order, dedup
        if e not in seen:
            seen.add(e); uniq.append(e)
    return uniq

def staircasetopology2d_qc(nx, ny):
    """
    Build a QuantumCircuit that places a fresh SU(4) gate on every edge
    of the 2-D staircase topology for an nx × ny grid (row-major indexing).
    """
    nqubits = nx * ny
    qc = QuantumCircuit(nqubits)
    for k, (q1, q2) in enumerate(_staircase_edges(nx, ny)):  # 1-based
        mat = random_su4()
        gate = UnitaryGate(mat, label=f"SU4_{k}")   # create the gate
        gate._name = "su4"                          # override its name
        qc.append(gate, [q1-1, q2-1])               # 0-based
    return qc

# Example usage:
nx, ny = 5,5
qc_2d = staircasetopology2d_qc(nx, ny)

# fig = qc_2d.draw(output="mpl", fold=-1)   # no line-wrapping
# plt.show()

In [ ]:
n = qc_2d.num_qubits # Here n is number of qubit in 2d circuit

plus_label = "+" * n # build a string of "+" of length n, e.g. "+++++"

# ─── Build the single-qubit X observable on qubit 0 ───────────────────────────
pauli_label = "X" + "I" * (n - 1) # For X on qubit 0 and Identity elsewhere, we need "X" + "I"*(n-1)
obs = Pauli(pauli_label)

# ─── Evolve the statevector through the 2D staircase circuit ─────────────────
psi0 = Statevector.from_label(plus_label)
psi_final = psi0.evolve(qc_2d)

# ─── Compute and print the expectation value ───────────────────────────────────
exp_val = psi_final.expectation_value(obs).real
print(f"⟨X on qubit 0⟩ from statevector simulation: {exp_val:.6f}")

In [11]:
# 5*5, 1 layer, k = 3, 12s
# 5*5, 1 layer, 

In [12]:
# 感觉精度变差了，尝试提高精度

prop_2d   = PauliPropagator(qc_2d)
print('1')

# 把字符串 → qiskit.Pauli → 位掩码
key_obs = encode_pauli(Pauli(pauli_label))

# 新构造方式：PauliTerm(coeff, key, n)
obs_term = PauliTerm(1.0, key_obs, qc_2d.num_qubits)

# 反向传播（可选 max_weight）
layers_2d = prop_2d.propagate(obs_term, max_weight = 3)
print('2')
# 计算期望值
plus_label = "+" * qc_2d.num_qubits         # |+⟩^{⊗n}
expect_2d  = prop_2d.expectation_pauli_sum(layers_2d[-1], plus_label)
print('3')
print(expect_2d)

1
2
3
0.028912676043719256


In [13]:
# -0.2206212726355423

In [32]:

# %%
# ─────────────────────────────
#  对比测试
# ─────────────────────────────

num_qubits = 15
qc = QuantumCircuit(num_qubits)
random.seed(42)
for _ in range(20):
    qs = random.sample(range(num_qubits), 10)
    qc.cx(qs[0], qs[1])
    qc.cx(qs[2], qs[3])
    qc.cx(qs[4], qs[5])
    qc.cx(qs[6], qs[7])
    qc.cx(qs[8], qs[9])

    # ———— 给每个 qubit 都加 T ————
    for q in range(num_qubits):
        qc.t(q)

    # ———— 第二轮随机 CNOT ————
    qs2 = random.sample(range(num_qubits), 10)
    qc.cx(qs2[0], qs2[1])
    qc.cx(qs2[2], qs2[3])
    qc.cx(qs2[4], qs2[5])
    qc.cx(qs2[6], qs2[7])
    qc.cx(qs2[8], qs2[9])

prop = PauliPropagator(qc)

key = encode_pauli(Pauli("X" + "I"*(num_qubits-1)))
obs = PauliTerm(1.0, key, num_qubits)

layers = prop.propagate(obs)

# 验证 expectation
expv = prop.expectation_pauli_sum(layers[-1], "+"*num_qubits)
print("expectation:", expv)

NameError: name 'PauliPropagator' is not defined

In [5]:
import math

# Convert two Boolean lists (x, z) into a readable Pauli string like 'IXY'
def pauli_str(x, z):
    symbols = []
    for xi, zi in zip(x, z):
        if xi == 0 and zi == 0: symbols.append('I')
        elif xi == 1 and zi == 0: symbols.append('X')
        elif xi == 1 and zi == 1: symbols.append('Y')
        else:                     symbols.append('Z')
    return ''.join(symbols)

# Initial observable: IXI  (X on qubit-1)
terms = [(1.0, [0, 1, 0], [0, 0, 0])]

gates = [
    ("CX", 1, 2),   # CNOT  control-1  target-2   (Heisenberg backward)
    ("T",  2),
    ("T",  1),
    ("T",  0),
    ("CX", 0, 1)    # CNOT  control-0  target-1
]

print("Boolean-array version")
step = 0
print(f"step {step}: {[f'{coeff:+g} {pauli_str(x, z)}' for coeff, x, z in terms]}")

for gate in gates:
    step += 1
    new_terms = []
    for coeff, x, z in terms:
        if gate[0] == "CX":
            _, c, t = gate
            x_c, z_t = x[c], z[t]
            x2, z2 = x.copy(), z.copy()
            if x_c:
                x2[t] ^= 1          # X_t  -> X_t  XOR X_c
            if z_t:
                z2[c] ^= 1          # Z_c  -> Z_c  XOR Z_t
            new_terms.append((coeff, x2, z2))

        else:                       # T gate
            _, q = gate
            x_q, z_q = x[q], z[q]
            if (z_q and not x_q) or (not x_q and not z_q):
                new_terms.append((coeff, x.copy(), z.copy()))      # I or Z branch
            else:
                c1 = coeff / math.sqrt(2)
                c2 = -c1 if z_q else c1
                new_terms.append((c1, x.copy(), z.copy()))         # keep branch
                x2, z2 = x.copy(), z.copy()
                z2[q] ^= 1                                         # flip Z_q
                new_terms.append((c2, x2, z2))

    terms = new_terms
    print(f"step {step}: {[f'{coeff:+g} {pauli_str(x, z)}' for coeff, x, z in terms]}")


Boolean-array version
step 0: ['+1 IXI']
step 1: ['+1 IXX']
step 2: ['+0.707107 IXX', '+0.707107 IXY']
step 3: ['+0.5 IXX', '+0.5 IYX', '+0.5 IXY', '+0.5 IYY']
step 4: ['+0.5 IXX', '+0.5 IYX', '+0.5 IXY', '+0.5 IYY']
step 5: ['+0.5 IXX', '+0.5 ZYX', '+0.5 IXY', '+0.5 ZYY']


In [20]:
import math

# Convert two integer bit-masks (x_bits, z_bits) into a readable Pauli string
def pauli_str_bits(x_bits, z_bits, n_qubits=3):
    symbols = []
    for i in range(n_qubits):
        xi = (x_bits >> i) & 1
        zi = (z_bits >> i) & 1
        if xi == 0 and zi == 0: symbols.append('I')
        elif xi == 1 and zi == 0: symbols.append('X')
        elif xi == 1 and zi == 1: symbols.append('Y')
        else:                     symbols.append('Z')
    return ''.join(symbols)

# Initial observable: IXI  (X on qubit-1)
terms = [(1.0, 1 << 1, 0)]     # x_bits=0b010, z_bits=0

gates = [
    ("CX", 1, 2),
    ("T",  2),
    ("T",  1),
    ("T",  0),
    ("CX", 0, 1)
]

print("\nBit-mask version")
step = 0
print(f"step {step}: {[f'{coeff:+g} {pauli_str_bits(xb, zb)}' for coeff, xb, zb in terms]}")

for gate in gates:
    step += 1
    new_terms = []
    for coeff, xb, zb in terms:
        if gate[0] == "CX":
            _, c, t = gate
            x_c = (xb >> c) & 1
            z_t = (zb >> t) & 1
            xb2 = xb ^ ((1 << t) if x_c else 0)
            zb2 = zb ^ ((1 << c) if z_t else 0)
            new_terms.append((coeff, xb2, zb2))

        else:                       # T gate
            _, q = gate
            x_q = (xb >> q) & 1
            z_q = (zb >> q) & 1
            if (z_q and not x_q) or (not x_q and not z_q):
                new_terms.append((coeff, xb, zb))
            else:
                c1 = coeff / math.sqrt(2)
                c2 = -c1 if z_q else c1
                new_terms.append((c1, xb, zb))
                xb2, zb2 = xb, zb ^ (1 << q)
                new_terms.append((c2, xb2, zb2))

    terms = new_terms
    print(f"step {step}: {[f'{coeff:+g} {pauli_str_bits(xb, zb)}' for coeff, xb, zb in terms]}")



Bit-mask version
step 0: ['+1 IXI']
step 1: ['+1 IXX']
step 2: ['+0.707107 IXX', '+0.707107 IXY']
step 3: ['+0.5 IXX', '+0.5 IYX', '+0.5 IXY', '+0.5 IYY']
step 4: ['+0.5 IXX', '+0.5 IYX', '+0.5 IXY', '+0.5 IYY']
step 5: ['+0.5 IXX', '+0.5 ZYX', '+0.5 IXY', '+0.5 ZYY']


In [21]:
import math

def t_dag_branchless(terms, q):
    """
    Branchless conjugation by T† on qubit q.
    terms: list of (coeff, x_bits, z_bits)
    q: target qubit index (0-based)
    Returns: new list of (coeff, x_bits, z_bits)
    """
    new_terms = []
    inv_sqrt2 = 1 / math.sqrt(2)
    for coeff, x_bits, z_bits in terms:
        x_q = (x_bits >> q) & 1
        z_q = (z_bits >> q) & 1
        
        # Compute coefficients without branches
        c1 = coeff * ((1 - x_q) + x_q * inv_sqrt2)
        c2 = coeff * (x_q * ((2 * z_q - 1) * inv_sqrt2))
        
        # Only flip Z bit when x_q == 1
        zmask = x_q << q
        
        # Append both branches
        new_terms.append((c1, x_bits,        z_bits))
        new_terms.append((c2, x_bits, z_bits ^ zmask))
    
    return new_terms

# Demonstration for each P in {I, X, Y, Z} on qubit 0
test_terms = {
    'I': [(1.0, 0,        0       )],
    'X': [(1.0, 1 << 0,   0       )],
    'Y': [(1.0, 1 << 0,   1 << 0  )],
    'Z': [(1.0, 0,        1 << 0  )]
}

for label, terms in test_terms.items():
    result = t_dag_branchless(terms, 0)
    print(f"{label}: {result}")


I: [(1.0, 0, 0), (-0.0, 0, 0)]
X: [(0.7071067811865475, 1, 0), (-0.7071067811865475, 1, 1)]
Y: [(0.7071067811865475, 1, 1), (0.7071067811865475, 1, 0)]
Z: [(1.0, 0, 1), (0.0, 0, 1)]


In [23]:
test_terms = {
    'I': [(1.0, 0,        0       )],
    'X': [(1.0, 1 << 0,   0       )],
    'Y': [(1.0, 1 << 0,   1 << 0  )],
    'Z': [(1.0, 0,        1 << 0  )]
}

test_terms 


{'I': [(1.0, 0, 0)],
 'X': [(1.0, 1, 0)],
 'Y': [(1.0, 1, 1)],
 'Z': [(1.0, 0, 1)]}

In [11]:
 # 这是一个位运算表达式，表示将数字1左移1位
 # 结果等于2，因为1的二进制是0001，左移1位后变成0010，即十进制的2
1<<1

2